In [1]:
# This file consists of using built in native code executor of google ADK

In [2]:
# Importing basic libraries
from google.adk.agents import LlmAgent, SequentialAgent
from google.genai import types
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor
import os
from dotenv import load_dotenv

print("Libraries imported successfully")

/home/bot/repos/development/google-adk-practice/venv/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.12) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


Libraries imported successfully


In [3]:
# Checking configuration
google_api_key = os.getenv("GOOGLE_API_KEY")
if google_api_key:
    print("API KEY FOUND")
else:
    print("CONFIGURATION ERROR")

API KEY FOUND


In [4]:
# Retry config function
def create_retry_config():
    '''
    This function returns the basic retry configuration for the AI agent
    '''

    return types.HttpRetryOptions(
        attempts=3,
        initial_delay=1,
        exp_base=2,
        http_status_codes=[429, 500, 501, 503, 504]
    )

retry_config = create_retry_config()

In [5]:
# This helper functions prints the python code generated by the agent and the its generated result from the code executor tool

def show_python_code_and_result(response):
    for i in range(len(response)):
        # Checking if the response contains a valid function call result from the code executor
        if(
            (response[i].content.parts)
            and (response[i].content.parts[0])
            and (response[i].content.parts[0].function_response)
            and (response[i].content.parts[0].function_response.response)
        ):
            response_code = response[i].content.parts[0].function_response.response
            if "result" in response_code and response_code.result != "```":
                if "tool_code" in response_code["result"]:
                    print(
                        "Generated Python Code >> ",
                        response_code["result"].replace("tool_code", "")
                    )
                else:
                    print("Generated Python Response >> ", response_code["result"])

print("Defined helper function")

Defined helper function


In [ ]:
code_generator=LlmAgent(
    name="CodeGeneratorAgent",
    model="gemini-2.5-flash-lite",
    instruction='''You are an expert python code generator.
    Your task is to generate a python code snippet based on the input from the user..
    - ONLY output the generated code, no introduction or explaination
    ''',
    description="Python code generator agent that generates code based on the user request.",
    output_key="generated_code"
)

print("Code generator agent defined")

In [ ]:
code_executor=LlmAgent(
    name="CodeExecutorAgent",
    model="gemini-2.5-flash-lite",
    instruction='''You are a code executor.
    - Execute the following code snippet {generated_code} with a example argument to demonstrate the working of the code.
    - Use the 'BuiltInCodeExecutor' tool to execute the code
    - ONLY output the generated response/returned value, without any description of code's working.
    ''',
    description="Runs the code snippet passed by the generator_agent and outputs the returned value as response",
    code_executor=BuiltInCodeExecutor(),
    output_key="executed_code"
)

print("Code executor agent defined")


In [ ]:
root_agent=SequentialAgent(
    name="CodeGeneratorPipeline",
    sub_agents=[code_generator, code_executor],
)

runner = InMemoryRunner(agent=root_agent)

print("Root agent and runner is defined")

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/home/bot/repos/development/google-adk-practice/venv/lib/python3.10/site-packages/google/adk/agents", which implies app name "agents".


In [9]:
response = await runner.run_debug(
    "Generate a code snippet to caluclate factorial of a number."
)


 ### Created new session: debug_session_id

User > Generate a code snippet to caluclate factorial of a number.
CodeGeneratorAgent > ```python
def factorial(n):
    """
    Calculates the factorial of a non-negative integer.

    Args:
        n: A non-negative integer.

    Returns:
        The factorial of n.
    """
    if not isinstance(n, int) or n < 0:
        raise ValueError("Input must be a non-negative integer")
    if n == 0:
        return 1
    else:
        result = 1
        for i in range(1, n + 1):
            result *= i
        return result

if __name__ == '__main__':
    num = 5
    print(f"The factorial of {num} is {factorial(num)}")

    num = 0
    print(f"The factorial of {num} is {factorial(num)}")

    try:
        num = -2
        print(f"The factorial of {num} is {factorial(num)}")
    except ValueError as e:
        print(e)

    try:
        num = 3.5
        print(f"The factorial of {num} is {factorial(num)}")
    except ValueError as e:
        print(e)

CodeExecutorAgent > The factorial of 5 is 120
The factorial of 0 is 1
Input must be a non-negative integer
Input must be a non-negative integer

